In [40]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [41]:
!pip install transformers
!pip install tensorflow==2.6.0
!pip install simpletransformers
!pip install tokenizers==0.11.1
!pip install texthero

In [42]:
!export CUDA_HOME=/usr/local/cuda-10.1
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

In [43]:
import pandas as pd

df = pd.read_csv("/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")
df.head()

Check for Missing Values

In [44]:
df.fillna(' ',inplace=True)

In [45]:
df['text'] = df['title'] + " " + df['department'] + " " + df['company_profile'] + " " + \
             df['description'] + " " + df['requirements'] + " " + df['benefits'] + " " +\
             df['function'] + " "

In [46]:
delete_list=['job_id','title','location','telecommuting','has_company_logo','has_questions','department','salary_range','company_profile','description','requirements','benefits','employment_type','required_experience','required_education','industry','function']

for val in delete_list:
    del df[val]
df.head()

In [47]:
import spacy, re
#Data Cleanup

df['text']=df['text'].str.replace('\n','')
df['text']=df['text'].str.replace('\r','')
df['text']=df['text'].str.replace('\t','')
  
#This removes unwanted texts
df['text'] = df['text'].apply(lambda x: re.sub(r'[0-9]','',x))
df['text'] = df['text'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))
  
#Converting all upper case to lower case
df['text']= df['text'].apply(lambda s:s.lower() if type(s) == str else s)

#Remove un necessary white space
df['text']=df['text'].str.replace('  ',' ')

#Remove Stop words
nlp=spacy.load("en_core_web_sm")
df['text'] =df['text'].apply(lambda x: ' '.join([word for word in x.split() if nlp.vocab[word].is_stop==False ]))

# text ['remove_urls']=hero.remove_urls(text ['remove_stopwords'])
# text ['remove_htmls']=hero.remove_html_tags(text ['remove_urls'])

!pip install texthero
import texthero as hero
clean_pipeline = [hero.preprocessing.remove_digits,
                    hero.preprocessing.remove_diacritics,
                    hero.preprocessing.remove_stopwords,
                    hero.preprocessing.remove_urls,
                    hero.remove_html_tags,
                    hero.remove_punctuation,
                    hero.preprocessing.remove_whitespace,
                    hero.preprocessing.fillna
                   ]
df['text'] = df['text'].pipe(hero.clean, clean_pipeline)

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df.fraudulent, test_size = 0.20, stratify=df.fraudulent, random_state=777)

train_df = pd.DataFrame({0: X_train, 1: y_train})
test_df = pd.DataFrame({0: X_test, 1: y_test})

In [49]:
from simpletransformers.classification import ClassificationModel


model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, 
                            args={'overwrite_output_dir': True, 
                                  "train_batch_size": 64, 
                                  "save_steps": 10000, 
                                  "save_model_every_epoch":False,
                                  'num_train_epochs': 10}, use_cuda=True)

In [50]:
model.train_model(train_df)

In [51]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [52]:
import numpy as np
preds = [np.argmax(tuple(m)) for m in model_outputs]

In [53]:
from sklearn.metrics import f1_score

print(f1_score(test_df[1], preds, average='micro'))
print(f1_score(test_df[1], preds, average='macro'))

In [54]:
from sklearn.metrics import classification_report

print(classification_report(test_df[1], preds))

In [55]:
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
test_classes = test_df[1] #np.array(y_test, dtype='int')
cm = confusion_matrix(test_classes, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

[[tn, fp], [fn, tp]] =cm


accuracy = (tp+tn)/(tn+fp+fn+tp)
precision = tp/(tp+fp)
TPR = tp/(tp+fn)
TNR = tn/(tn+fp)

print('accuracy={}'.format(accuracy))
print('precision={}'.format(precision))
print('TPR={}'.format(TPR))
print('TNR={}'.format(TNR))

In [56]:
# classifier_roc_auc = roc_auc_score(test_classes, preds)
fpr, tpr, thresholds = roc_curve(test_classes, preds)
classifier_roc_auc, fpr, tpr, thresholds